In [1]:
# 107303538_1223_hw14_2

In [2]:
import numpy as np 
import torch
import torch.nn as nn
from matplotlib import pyplot as plt

In [3]:
from google.colab import drive # 連接到google drive
drive.mount('/content/drive')

Mounted at /content/drive


匯入labels.csv

In [4]:
path = './drive/MyDrive/Dog/labels.csv' 
dogs_numpy = np.loadtxt(path, dtype=str, delimiter=',', skiprows=1) # 處理 excel有內建分隔delimiter 去除第一行(id breed)
dogs_numpy, dogs_numpy.shape

(array([['000bec180eb18c7604dcecc8fe0dba07', 'boston_bull'],
        ['001513dfcb2ffafc82cccf4d8bbaba97', 'dingo'],
        ['001cdf01b096e06d78e9e5112d419397', 'pekinese'],
        ...,
        ['ffe2ca6c940cddfee68fa3cc6c63213f', 'airedale'],
        ['ffe5f6d8e2bff356e9482a80a6e29aac', 'miniature_pinscher'],
        ['fff43b07992508bc822f33d8ffd902ae', 'chesapeake_bay_retriever']],
       dtype='<U32'), (10222, 2))

匯入breed.txt

In [5]:
path = './drive/MyDrive/Dog/breed.txt' 
breed_numpy = np.loadtxt(path, dtype=str)
#breed_numpy

In [6]:
# 轉成list
breed_list = list() # (可略) 空list為list()或[]
breed_list = list(breed_numpy)
#breed_list

In [7]:
# 轉成dic
dic = {} # (不可略) 空dict為dict()或{}
for i in range(len(breed_list)):
    dic[breed_list[i]] = i # dic[ key ] = value
#dic

處理dog_numpy(labels.csv)

In [8]:
dog_labels = list(dogs_numpy[:,-1]) # dogs_numpy為(10222, 2) 10222總狗 橫向看為(id, breed) [:,-1]為取所有狗的breed而已
dog_labels = [dic[breed_name] for breed_name in dog_labels] # dic[breed_name]最後會產出value 即找dog_labels內對應到dict的標號
dog_labels # 拚加成list dic[]已經是一個裝有breed_list的字典了 有10222個 算是10222個不同圖片的正確答案
dog_t_labels = torch.tensor(dog_labels) # 轉成tensor
dog_t_labels

# 舉例: dogs_numpy的第一種狗'boston_bull' 在breed.txt中為第19個 故dic['boston_bull']=19 而dog_labels[0]和dog_t_labels[0]為19和tensor(19)

tensor([19, 37, 85,  ...,  3, 75, 28])

匯入train圖片

In [9]:
import os
dog_file_name = [i +'.jpg' for i in dogs_numpy[:,0]] # dogs_numpy[:,0]為id dog_file_name為list
dog_file_path = [os.path.join('./drive/MyDrive/Dog/train', i_jpg) # 取train中每一個圖片 os.path.join為合併路徑
          for i_jpg in dog_file_name] 
#dog_file_path

設定資料集

前處理preprocess(含正規化)

In [10]:
from torchvision import transforms, utils

preprocess = transforms.Compose([transforms.Resize([24, 24]), # 變正方形
                  transforms.ToTensor(),
                  transforms.Normalize(mean=[0.4766, 0.4523, 0.3923], std=[0.2205, 0.2154, 0.2137]) # 正規化
])

定義loader

In [11]:
# 輸入path返回tensor
from PIL import Image # 匯入外部圖片
def default_loader(path):
    img_pil = Image.open(path) # 打開真正的圖片
    img_tensor = preprocess(img_pil) # 處理圖片
    return img_tensor

定義Dataset

In [12]:
# 定義Dataset類別 (init / getitem / len)
from torch.utils.data import Dataset, DataLoader

class TrainSet(Dataset):
  def __init__(self, load=default_loader): # 初始化
      self.images = dog_file_path # dog_file_path為照片的完整路徑
      self.target = dog_t_labels # 真正的狗的labels 正確答案
      self.loader = default_loader
  
  def __getitem__(self, index): # 內建getitem的規則(有點像是打開一張圖片的方法)
      fn = self.images[index] # 也就是dog_file_path[1]
      img = self.loader(fn) # 輸出真正的圖片 並前處理後 返回其tensor
      target = self.target[index] # dog_t_labels[1] 真正的labels
      return img, target

  def __len__(self):
    return len(self.images) # 資料集長度

In [13]:
train_data = TrainSet() # 把model改名

定義train_loader

In [14]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)  # DataLoader

設定model

ResBlock(重複區)

In [15]:
import torch.nn.functional as F

class ResBlock(nn.Module): # ResBlock
  def __init__(self, n_chans): 
    super(ResBlock, self).__init__() 
    # conv
    self.conv = nn.Conv2d(n_chans, n_chans, kernel_size=3, padding=1, bias=False) # 不需要bias 因為後面會normalize
    
    # norm
    self.batch_norm = nn.BatchNorm2d(num_features=n_chans) # BatchNorm2d用於2D 以minibatch為單位正規化(有bias+weight)
    torch.nn.init.constant_(self.batch_norm.weight, 0.5) # batch的weight初始為0.5(常數)
    torch.nn.init.zeros_(self.batch_norm.bias) # batch的bias初始為0
    
    # relu
    torch.nn.init.kaiming_normal_(self.conv.weight, nonlinearity='relu') # kaiming方法去normal relu為和tahn一樣的激活神經網路
    
  def forward(self, x): # ResBlock執行順序 : conv + batch_norm + relu(神經網路)
    out = self.conv(x)
    out = self.batch_norm(out)
    out = torch.relu(out) 
    return out + x # out是訓練完的 x是未訓練的

NetResDeep model(一般區)

In [26]:
class NetResDeep(nn.Module):
  def __init__(self, n_chans1=16, n_chans2=6, n_blocks=10):
    super().__init__()
    #self.n_chans1 = n_chans1
    self.n_chans2 = n_chans2

    self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
    self.conv2 = nn.Conv2d(n_chans1, n_chans2, kernel_size=3, padding=1)
    self.resblocks = nn.Sequential(*(n_blocks * [ResBlock(n_chans=n_chans2)])) # n_blocks為做幾次跌代 nn.Sequential會自動調整大小並跌

    self.fc1 = nn.Linear(6 * 6 * n_chans2, 180) 
    self.fc2 = nn.Linear(180, 120)

  def forward(self, x):
    out = F.max_pool2d(torch.relu(self.conv1(x)), 2) # 經過tahn神經網路 再縮小成二分之一 
    out = F.max_pool2d(torch.relu(self.conv2(out)), 2)
    out = self.resblocks(out) # 做n_block次的resblocks

    out = out.view(-1, 6 * 6 * self.n_chans2) 
    out = torch.relu(self.fc1(out)) # 變扁平去訓練後 再進去一個relu
    out = self.fc2(out)
    return out

In [27]:
model = NetResDeep() # model改名

訓練一張圖片

In [28]:
img_1_t = default_loader(dog_file_path[0]) # 第一張圖片的tensor
img_1_t = img_1_t.unsqueeze(0) # 變成可以訓練的格式 torch.Size([1, 3, 32, 32]) ( B, C, H, W )
out = model(img_1_t)

In [29]:
_, index = torch.max(out, dim=1) # 分類 機率中較大的 即為越像該物
index # model猜的

dogname = breed_numpy[index] # 對應到的狗名
dogname # 預測的狗名字

'gordon_setter'

設定訓練參數

optimizer + loss_fn

In [30]:
import torch.optim as optim # 最佳化資料庫

optimizer = optim.SGD(model.parameters(), lr=1e-3) 
loss_fn = nn.CrossEntropyLoss() # 計算實際輸出(機率)與期望輸出(機率)的距離

In [31]:
# 學太慢改善方式 : 學習率調大 batch調大

設定train loop

In [32]:
import datetime # 計算時間用

def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
  for epoch in range(1, n_epochs + 1): 
    loss_train = 0.0 # 設定loss
    
    for imgs, target in train_loader:
      outputs = model(imgs) # 訓練 得出機率 outputs為tensor(1,120)
      loss = loss_fn(outputs, target) # 訓練機率和真正答案比較

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      loss_train += loss.item() # loss_train為累加loss 有.item()就可以跳離grad

    if epoch == 1 or epoch % 5 == 0: 
      print('{} Epoch {}, Training loss {}'.format(
        datetime.datetime.now(), epoch,
        loss_train / len(train_loader))) # 除以loss次數

train loop

In [ ]:
training_loop(
  n_epochs = 100,
  optimizer = optimizer,
  model = model,
  loss_fn = loss_fn,
  train_loader = train_loader
)